#### Distance Metrics to be Tested

1. Cosine 
2. Euclidean
3. Yule

In [ ]:
#'''Required Libraries (uncomment for installation)'''

# !pip install pandas
# !pip install numpy
# !pip install scipy

In [27]:
'''Importing Libraries'''

import os
import pandas as pd
import numpy as np
from scipy.spatial import distance

#### Distance Calulations

In [76]:
'''
Calculating Ditance Metrics
'''

def after_distance(cont_arr, optio):
    
    max_arr, idx2, word = [], [], []
    
    for instances in range(len(cont_arr)):
        max_arr.append(np.amax(cont_arr[instances]))
    
    for ind_val2 in range(len(max_arr)):
        idx2.append(np.where(cont_arr[ind_val2] == max_arr[ind_val2]))
    idx2 = np.asarray(idx2)

    for i in range(len(optio)):
            word.append(optio[i][idx2[i]])
        
    return max_arr, word

In [96]:
'''
Calculating Distance Function
'''

def cal_distances(quest, optio, model="LSA"):    
    cosi_arr, eucd_arr, yule_arr = [], [], []
          
    for i in range(len(quest)):
        tmp_arr_cosi, tmp_arr_eucd, tmp_arr_yule = [], [], []
        
        for j in range(len(optio[i])):
            
            if model == "LSA":
                # Calculating Cosine 
                tmp_arr_cosi.append(1 - (distance.cosine(dict_word_vectors[quest[i][0]] - dict_word_vectors[quest[i][2]], \
                                                         dict_word_vectors[quest[i][1]] - dict_word_vectors[optio[i][j]])))

                # Calculating Euclidean 
                tmp_arr_eucd.append(1 - (distance.euclidean(dict_word_vectors[quest[i][0]] - dict_word_vectors[quest[i][2]], \
                                                            dict_word_vectors[quest[i][1]] - dict_word_vectors[optio[i][j]])))

                # Calculating Yule
                tmp_arr_yule.append(1 - (distance.yule(dict_word_vectors[quest[i][0]] - dict_word_vectors[quest[i][2]], \
                                                   dict_word_vectors[quest[i][1]] - dict_word_vectors[optio[i][j]])))
            
            elif model == "GloVe":
                # Calculating Cosine 
                tmp_arr_cosi.append(1 - (distance.cosine(glove_word_vector[quest[i][0]] - glove_word_vector[quest[i][2]], \
                                                         glove_word_vector[quest[i][1]] - glove_word_vector[optio[i][j]])))

                # Calculating Euclidean 
                tmp_arr_eucd.append(1 - (distance.euclidean(glove_word_vector[quest[i][0]] - glove_word_vector[quest[i][2]], \
                                                            glove_word_vector[quest[i][1]] - glove_word_vector[optio[i][j]])))

                # Calculating Yule
                tmp_arr_yule.append(1 - (distance.yule(glove_word_vector[quest[i][0]] - glove_word_vector[quest[i][2]], \
                                                   glove_word_vector[quest[i][1]] - glove_word_vector[optio[i][j]])))
                
        
        # Appending Arrays
        cosi_arr.append(tmp_arr_cosi)
        eucd_arr.append(tmp_arr_eucd)
        yule_arr.append(tmp_arr_yule)
    
    
    max_arr_cosi, word_cosi = after_distance(cosi_arr, optio)
    max_arr_eucd, word_eucd = after_distance(eucd_arr, optio)
    max_arr_yule, word_yule = after_distance(yule_arr, optio)
    
    # Generating Dataframes
    cosi_df = pd.DataFrame({"Calculated":word_cosi, "Similarity": max_arr_cosi, "Actual":ans_list})
    eucd_df = pd.DataFrame({"Calculated":word_eucd, "Similarity": max_arr_eucd, "Actual":ans_list})
    yule_df = pd.DataFrame({"Calculated":word_yule, "Similarity": max_arr_yule, "Actual":ans_list})
    
    return cosi_df, eucd_df, yule_df

#### Calling Questions and Options Dataframes for list Conversions 

In [58]:
'''Create List'''

def conv_list(df):
    que_list, opt_list = [], []
    
    for idx, row in df.iterrows():
        tmp_list_que = [row.A, row.B, row.C]
        tmp_list_opt = [row.E, row.F, row.G, row.H]

        que_list.append(tmp_list_que)
        opt_list.append(tmp_list_opt)

    return np.asarray(que_list), np.asarray(opt_list)

In [59]:
question = pd.read_csv('Que_Opt.csv', index_col=0)
answers  = pd.read_csv('Answers.csv', index_col=0)

ans_list = list(answers.T)
que_list, opt_list = conv_list(question)


### Importing Models

#### LSA .txt files for different k values

In [60]:
# # For Single File Tests

# '''Importing pickle txt File'''

# import pickle

# # Provide File Path
# path = "fiction_term_embeddings_TXT/term2topic_fiction_2.txt"

# word_file = open(path, "rb")
# word_vectors = pickle.load(word_file)
# word_file.close()

# dict_word_vectors = word_vectors
# dict_word_vectors

In [103]:
# For fictional genere and all k values runs

'''Importing all txt File and converting into '''

import pickle

k_arr = ["2", "4", "8", "16", "32", "64"]
genre_arr = ["fiction", "nonfiction", "poetry"]      # Make changes in the loop - currently set to genre_arr[0] as default

for i in range(len(genre_arr)):
    for j in range(len(k_arr)):
        
        ## Calling LSA model
        path = genre_arr[0] + "_term_embeddings_TXT/term2topic_" + genre_arr[0] + "_" + k_arr[j] + ".txt"
        
        word_file = open(path, "rb")
        word_vectors = pickle.load(word_file)
        word_file.close()

        dict_word_vectors = word_vectors
        
        cosi_similarity, eucd_similarity, yule_similarity = cal_distances(que_list, opt_list, "LSA")
        
        # Displaying Similarity Scores
        
        display(cosi_similarity.style.set_caption("Cosine Similarity Scores for " + genre_arr[i] + " at " + "k = " + k_arr[j]))
        display(eucd_similarity.style.set_caption("Euclidean Similarity Scores for " + genre_arr[i] + " at " + "k = " + k_arr[j]))
        display(yule_similarity.style.set_caption("Yule Similarity Scores for " + genre_arr[i] + " at " + "k = " + k_arr[j]))
        
        

,Calculated,Similarity,Actual
0,[['night']],0.982207,duck
1,[['fell']],0.803262,squash
2,[['london']],-0.949074,damage
3,[['ascent']],0.997600,ascent
4,[['imprisonment']],0.927546,render


,Calculated,Similarity,Actual
0,[['night']],0.978548,duck
1,[['fell']],0.940748,squash
2,[['london']],0.982798,damage
3,[['absence']],0.997322,ascent
4,[['imprisonment']],0.995663,render


,Calculated,Similarity,Actual
0,[['thrills']],0.749736,duck
1,[['fell']],0.968250,squash
2,[['london']],1.023529,damage
3,[['earth']],3.617611,ascent
4,[['foot']],1.636393,render


,Calculated,Similarity,Actual
0,[['night']],0.845297,duck
1,[['fell']],0.582123,squash
2,[['london']],-0.813826,damage
3,[['ascent']],0.906467,ascent
4,[['imprisonment']],0.874400,render


,Calculated,Similarity,Actual
0,[['night']],0.970486,duck
1,[['fell']],0.938502,squash
2,[['london']],0.974057,damage
3,[['absence']],0.986437,ascent
4,[['imprisonment']],0.991560,render


...

,Calculated,Similarity,Actual
0,[['storm']],0.901785,duck
1,[['curate']],0.882224,squash
2,[['thunder']],0.944803,damage
3,[['ascent']],0.929814,ascent
4,[['imprisonment']],0.969012,render


,Calculated,Similarity,Actual
0,[['storm']],1.369913,duck
1,[['fell']],1.007682,squash
2,[['exodus']],0.940680,damage
3,[['ascent']],0.866999,ascent
4,[['days']],2.861014,render


,Calculated,Similarity,Actual
0,[['night']],0.239914,duck
1,[['fell']],-0.238132,squash
2,[['thunder']],-0.139140,damage
3,[['child']],0.039000,ascent
4,[['imprisonment']],0.240854,render


,Calculated,Similarity,Actual
0,[['storm']],0.877958,duck
1,[['curate']],0.862810,squash
2,[['thunder']],0.933902,damage
3,[['ascent']],0.911866,ascent
4,[['imprisonment']],0.954184,render


,Calculated,Similarity,Actual
0,[['storm']],8.036416,duck
1,[['weybridge']],1.002364,squash
2,[['london']],0.863030,damage
3,[['child']],2.470902,ascent
4,[['days']],2.173379,render


#### Glove 

In [99]:
## Parsing File for non-LSA models

def parse_file(file_name):
    word_vectors = {}
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            first_whitespace = line.index(" ")
            word = line[:first_whitespace].strip('"')
            vector = np.array(line[first_whitespace + 1:].split(" "), dtype=np.float32)
            word_vectors[word] = vector
            
    return word_vectors

In [108]:
glove_arr = ["50", "100", "200", "300"]

## Calling Glove Model

for i in range(len(genre_arr)):
    for j in range(len(glove_arr)):
        
        path = "glove.6B/glove.6B." + glove_arr[i] + "d.txt"
        glove_word_vector = parse_file(path)
        
        cosi_similarity_glove, eucd_similarity_glove, yule_similarity_glove = cal_distances(que_list, opt_list, "GloVe")

        # Displaying Similarity Scores

        display(cosi_similarity_glove.style.set_caption("Glove Cosine Similarity Scores for " + genre_arr[i] + " using Glove at " + glove_arr[j]))
        display(eucd_similarity_glove.style.set_caption("Glove Euclidean Similarity Scores for " + genre_arr[i] + " using Glove at " + glove_arr[j]))
        display(yule_similarity_glove.style.set_caption("Glove Yule Similarity Scores for " + genre_arr[i] + " using Glove at " + glove_arr[j]))



,Calculated,Similarity,Actual
0,[['night']],0.411769,duck
1,[['fell']],0.089228,squash
2,[['thunder']],0.022336,damage
3,[['ascent']],0.126226,ascent
4,[['foot']],0.104647,render


,Calculated,Similarity,Actual
0,[['night']],-5.139254,duck
1,[['fell']],-7.569297,squash
2,[['thunder']],-7.273635,damage
3,[['ascent']],-7.467527,ascent
4,[['days']],-4.863820,render


,Calculated,Similarity,Actual
0,[['storm']],1.021921,duck
1,[['curate']],2.846515,squash
2,[['outwit']],4.004144,damage
3,[['child']],1.225594,ascent
4,[['foot']],0.631164,render


,Calculated,Similarity,Actual
0,[['night']],0.411769,duck
1,[['fell']],0.089228,squash
2,[['thunder']],0.022336,damage
3,[['ascent']],0.126226,ascent
4,[['foot']],0.104647,render


,Calculated,Similarity,Actual
0,[['night']],-5.139254,duck
1,[['fell']],-7.569297,squash
2,[['thunder']],-7.273635,damage
3,[['ascent']],-7.467527,ascent
4,[['days']],-4.863820,render


...

,Calculated,Similarity,Actual
0,[['night']],-8.216085,duck
1,[['fell']],-9.532506,squash
2,[['thunder']],-10.130370,damage
3,[['absence']],-10.609083,ascent
4,[['days']],-7.003567,render


,Calculated,Similarity,Actual
0,[['thrills']],0.799622,duck
1,[['curate']],1.104296,squash
2,[['outwit']],1.064845,damage
3,[['child']],1.209616,ascent
4,[['imprisonment']],117.038647,render


,Calculated,Similarity,Actual
0,[['night']],0.323546,duck
1,[['fell']],0.086107,squash
2,[['thunder']],0.024712,damage
3,[['ascent']],0.088820,ascent
4,[['days']],0.034136,render


,Calculated,Similarity,Actual
0,[['night']],-8.216085,duck
1,[['fell']],-9.532506,squash
2,[['thunder']],-10.130370,damage
3,[['absence']],-10.609083,ascent
4,[['days']],-7.003567,render


,Calculated,Similarity,Actual
0,[['thrills']],0.799622,duck
1,[['curate']],1.104296,squash
2,[['outwit']],1.064845,damage
3,[['child']],1.209616,ascent
4,[['imprisonment']],117.038647,render


#### MDS